This is the system which I have created as a part of the capstone project while doing PG certification from SP jain School of Global Management, Mumbai. (Oct 2016)

### Import the required libraries

In [1]:
import pandas as pd
import operator
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
from gensim import corpora, models, similarities
from gensim.corpora import TextCorpus, MmCorpus, Dictionary

from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import RegexpTokenizer
from string import digits
import os
import pprint

C:\Users\d.sadashiv.shinde\AppData\Local\Continuum\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
2017-08-16 19:43:18,011 : INFO : 'pattern' package not found; tag filters are not available for English


### set path

In [2]:
os.chdir('C:\\Users\\d.sadashiv.shinde\\Desktop\\ACIT Code\\NLP-QnASystem-master\\')
folderDestination = os.path.join(os.getcwd(),"Sample Text.csv")

### get data and convert to string

In [4]:
rAw = pd.read_csv("Sample Text.csv",sep = ',',header = 0, encoding='latin-1')
print(rAw)
rAw_ls = rAw.review.tolist()
'''to print first 5 rows of reviews'''
#rAw = rAw.ix[1:5,'review']
#rAw_ls = rAw.tolist()
#print(rAw.tolist())
# convert all elements to strings
rAw_ls = [str(i) for i in rAw_ls] 
print(rAw_ls)

                                                 review
0     I'm not a huge mcds lover, but I've been to be...
1     Terrible customer service. .I came in at 9:30p...
2     First they "lost" my order, actually they gave...
3     I see I'm not the only one giving 1 star. Only...
4     Well, it's McDonald's, so you know what the fo...
5     This has to be one of the worst and slowest Mc...
6     I'm not crazy about this McDonald's. .This is ...
7     One Star and I'm beng kind. I blame management...
8     Never been upset about any fast food drive thr...
9     This McDonald's has gotten much better. Usuall...
10    Let's start here only reason I came into McDon...
11    Other businesses throughout Metro Atlanta open...
12    The drive thru makes them lost a star since my...
13    Believe it or not, this used to be q really go...
14    As the previous yelpers have already stated, t...
15    25 minutes in drive through line. Gunshots fro...
16    Super slow service, food's terrible like i

### Cleaning the raw text
Preprocessing of the raw text includes 
- lemmatizing , 
- removal of special characters
- removal of numbers, 
- English stopwords, 
- basis special characters 
- and removing words having frequency of 1

In [9]:
''' Preprocessing of the raw text includes lemmatizing , removal of special characters
removal of numbers, English stopwords, tokenizing basis special characters and removing 
words having frequency of 1'''
def preProcess(lst):
    lemmatizer = WordNetLemmatizer()
    stop_words = stopwords.words("english")
    #stop_words.extend(custom_stopW)
    stop_words = set(stop_words)
    punc = RegexpTokenizer(r'((?<=[^\w\s])\w(?=[^\w\s])|(\W))+', gaps=True)
    doc = [word.lower() for word in lst]
    words = [punc.tokenize(text) for text in doc] # tokenize basis punctuations
    specialChar = [",","."," ","?","!","'"] #  remove special characthers
    words = [[term for term in word if term not in specialChar] for word in words]
    words_filter = [[word for word in document if word not in stop_words]
                        for document in words]      
    words_nodig = [[word.strip(digits) for word in document]
                        for document in words_filter]
    words_nodig = [[word for word in document if  word !=" "]
                        for document in words_nodig]
    words_stem = [[lemmatizer.lemmatize(word) for word in document]
                        for document in words_nodig]
    # remove words that appear only once
    from collections import defaultdict
    frequency = defaultdict(int)
    for document in words_stem:
        for token in document:
            frequency[token] += 1
    
    words_final = [[token for token in document if frequency[token] > 1]
                        for document in words_stem]
	#print(words_final)
    return words_final

cLeaned = preProcess(rAw_ls)
print(cLeaned)


[['huge', 'mcds', 'lover', 'better', 'one', 'far', 'worst', 'one', 'ever', 'filthy', 'inside', 'get', 'drive', 'completely', 'screw', 'order', 'every', 'time', 'staff', 'terribly', 'unfriendly', 'nobody', 'seems', 'care'], ['terrible', 'customer', 'service', 'came', '', ':', ':', 'pm', 'stood', 'front', 'register', 'one', 'bothered', 'say', 'anything', 'help', '', 'minute', 'one', 'else', 'waiting', 'food', 'inside', 'either', 'outside', 'window', 'left', 'went', 'next', 'door', 'greeted', 'way', 'inside', 'mcdonalds', 'also', 'dirty', 'floor', 'covered', 'dropped', 'food', 'obviously', 'filled', 'surly', 'unhappy', 'worker'], ['first', '"', '"', 'lost', 'order', 'actually', 'gave', 'someone', 'one', 'else', 'took', '', 'minute', 'figure', 'still', 'waiting', 'order', 'asked', 'needed', 'replied', '"', '"', 'order', 'asked', 'ticket', 'mgr', 'looked', 'ticket', 'filled', 'ask', 'check', 'see', 'filled', 'correctly', 'acted', 'bothered', 'asked', 'begrudgingly', 'checked', 'fact', 'miss

## Create Corpus

In [10]:
class myCorpus1(object):
    '''Create Dictionary & Corpus the dictionary and corpus is getting saved on disk and hence it is agnostic of RAM'''
    def __init__(self,cLeaned,dictionary):
        self.cLeaned = cLeaned
        self.dictionary = dictionary
    def __iter__(self):
        for text in self.cLeaned:
            yield self.dictionary.doc2bow(text)

In [11]:
class myCorpus(object):
    '''Create Dictionary & Corpus the dictionary and corpus is getting saved on disk and hence it is agnostic of RAM'''
    def __init__(self,lst):
        self.lst = lst
                        
    def createCorpus(self,dictPath,corpusPath):
        cLeaned = preProcess(self.lst)
        dictionary = corpora.Dictionary(word for word in cLeaned)
        dictionary.save(dictPath)
        corpus_memory_friendly = myCorpus1(cLeaned,dictionary)
        corpora.MmCorpus.serialize(corpusPath, corpus_memory_friendly)

In [12]:
corpus1 = myCorpus(rAw_ls)
print(corpus1)

## Create Similarity Matrix

In [13]:
corpus1.createCorpus('mcd.dict','mcd.mm')
corpus = corpora.MmCorpus('mcd.mm')
print(corpus[0])

2017-08-16 19:44:08,370 : INFO : adding document #0 to Dictionary(0 unique tokens: [])
2017-08-16 19:44:08,463 : INFO : built Dictionary(3763 unique tokens: ['huge', 'mcds', 'lover', 'better', 'one']...) from 1525 documents (total 73897 corpus positions)
2017-08-16 19:44:08,463 : INFO : saving Dictionary object under mcd.dict, separately None
2017-08-16 19:44:08,479 : INFO : saved mcd.dict
2017-08-16 19:44:08,481 : INFO : storing corpus in Matrix Market format to mcd.mm
2017-08-16 19:44:08,484 : INFO : saving sparse matrix to mcd.mm
2017-08-16 19:44:08,486 : INFO : PROGRESS: saving document #0
2017-08-16 19:44:08,614 : INFO : PROGRESS: saving document #1000
2017-08-16 19:44:08,679 : INFO : saved 1525x3763 matrix, density=1.032% (59233/5738575)
2017-08-16 19:44:08,682 : INFO : saving MmCorpus index to mcd.mm.index
2017-08-16 19:44:08,691 : INFO : loaded corpus index from mcd.mm.index
2017-08-16 19:44:08,694 : INFO : initializing corpus reader from mcd.mm
2017-08-16 19:44:08,697 : INFO :

[(0, 1.0), (1, 1.0), (2, 1.0), (3, 1.0), (4, 2.0), (5, 1.0), (6, 1.0), (7, 1.0), (8, 1.0), (9, 1.0), (10, 1.0), (11, 1.0), (12, 1.0), (13, 1.0), (14, 1.0), (15, 1.0), (16, 1.0), (17, 1.0), (18, 1.0), (19, 1.0), (20, 1.0), (21, 1.0), (22, 1.0)]


In [14]:

class BuildSimMatrix(object):
    
    def __init__(self,corpusPath,dictPath):
        self.corpus = corpora.MmCorpus(corpusPath)
        self.dict = corpora.Dictionary.load(dictPath)        

    def lsiSim(self,modelPath,indexPath):
        lsi = models.LsiModel(self.corpus, id2word=self.dict, num_topics=400)
        lsi.save(modelPath)        
        index = similarities.MatrixSimilarity(lsi[self.corpus]) # transform corpus to LSI space and index it
        index.save(indexPath)
        
    def tfIdfSim(self,modelPath,indexPath):
        tfidf = models.TfidfModel(self.corpus)
        tfidf.save(modelPath)
        index = similarities.MatrixSimilarity(tfidf[self.corpus]) # transform corpus to LSI space and index it
        index.save(indexPath)
        
    '''Random Projection indexing is an advanced method in retrival of  documents'''
    def randomProj(self,modelPath,indexPath):
        rp = models.RpModel(self.corpus)
        rp.save(modelPath)
        index = similarities.MatrixSimilarity(rp[self.corpus]) # transform corpus to LSI space and index it
        index.save(indexPath)

In [15]:
  
temp = BuildSimMatrix('mcd.mm','mcd.dict')
temp.lsiSim('lsi.model','lsiindex.index',)
temp.tfIdfSim('tfidf.model','tfidfindex.index')
temp.randomProj('rp.model','rpindex.index') 

2017-08-16 19:44:12,342 : INFO : loaded corpus index from mcd.mm.index
2017-08-16 19:44:12,344 : INFO : initializing corpus reader from mcd.mm
2017-08-16 19:44:12,347 : INFO : accepted corpus with 1525 documents, 3763 features, 59233 non-zero entries
2017-08-16 19:44:12,349 : INFO : loading Dictionary object from mcd.dict
2017-08-16 19:44:12,352 : INFO : loaded mcd.dict
2017-08-16 19:44:12,354 : INFO : using serial LSI version on this node
2017-08-16 19:44:12,356 : INFO : updating model with new documents
2017-08-16 19:44:12,524 : INFO : preparing a new chunk of documents
2017-08-16 19:44:12,541 : INFO : using 100 extra samples and 2 power iterations
2017-08-16 19:44:12,543 : INFO : 1st phase: constructing (3763, 500) action matrix
2017-08-16 19:44:12,621 : INFO : orthonormalizing (3763, 500) action matrix
2017-08-16 19:44:13,387 : INFO : 2nd phase: running dense svd on (500, 1525) matrix
2017-08-16 19:44:13,554 : INFO : computing the final decomposition
2017-08-16 19:44:13,554 : INFO 

## QueryQuestion class to fire a query on index

In [16]:
class QueryQuestion():
    def __init__(self):
        self.query = input('Type Query :')
            
    def lsi_query(self,indexPath,dictPAth,modelPath,orignaltext_list):
        index = similarities.MatrixSimilarity.load(indexPath)
        dict = corpora.Dictionary.load(dictPAth)
        lsi = models.LsiModel.load(modelPath)       
        vec_query = dict.doc2bow(self.query.lower().split())
        vec_lsi = lsi[vec_query]
        
        sims = index[vec_lsi] # perform a similarity query against the corpus
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        temp = [sims[x] for x in list(range(3))]
        temp_values  = [x[0] for x in temp]
        keys = [x[1] for x in temp]        
        values = [orignaltext_list[x] for x in temp_values]
        simDocs = { k:v for (k,v) in zip(keys, values)}
        return (simDocs)
    
    def tfidf_query(self,indexPath,dictPAth,modelPath,orignaltext_list):
        index = similarities.MatrixSimilarity.load(indexPath)
        dict = corpora.Dictionary.load(dictPAth)
        tfidf = models.LsiModel.load(modelPath)       
        vec_query = dict.doc2bow(self.query.lower().split())
        vec_tfidf = tfidf[vec_query]
        
        # perform a similarity query against the corpus
        sims = index[vec_tfidf] 
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        temp = [sims[x] for x in list(range(3))]
        temp_values  = [x[0] for x in temp]
        keys = [x[1] for x in temp]        
        values = [orignaltext_list[x] for x in temp_values]
        simDocs = { k:v for (k,v) in zip(keys, values)}
        return (simDocs)
    
    def rp_query(self,indexPath,dictPAth,modelPath,orignaltext_list):
        #LOAD DICT , INDEX AND MODEL
        index = similarities.MatrixSimilarity.load(indexPath)
        dict = corpora.Dictionary.load(dictPAth)
        rp = models.LsiModel.load(modelPath)          
        
        vec_query = dict.doc2bow(self.query.lower().split())       
        vec_rp = rp[vec_query]        
        # perform a similarity query against the corpus
        sims = index[vec_rp] 
        sims = sorted(enumerate(sims), key=lambda item: -item[1])
        temp = [sims[x] for x in list(range(3))]
        temp_values  = [x[0] for x in temp]
        keys = [x[1] for x in temp]        
        values = [orignaltext_list[x] for x in temp_values]
        simDocs = { k:v for (k,v) in zip(keys, values)}
        return (simDocs)

## LSI Query

In [17]:
temp_Q = QueryQuestion()
result_lsi = temp_Q.lsi_query('lsiindex.index','mcd.dict','lsi.model',rAw_ls)
pprint.pprint(result_lsi)

Type Query :macdonald


2017-08-16 19:45:13,468 : INFO : loading MatrixSimilarity object from lsiindex.index
2017-08-16 19:45:13,490 : INFO : loaded lsiindex.index
2017-08-16 19:45:13,492 : INFO : loading Dictionary object from mcd.dict
2017-08-16 19:45:13,497 : INFO : loaded mcd.dict
2017-08-16 19:45:13,499 : INFO : loading LsiModel object from lsi.model
2017-08-16 19:45:13,506 : INFO : loading id2word recursively from lsi.model.id2word.* with mmap=None
2017-08-16 19:45:13,510 : INFO : setting ignored attribute projection to None
2017-08-16 19:45:13,512 : INFO : setting ignored attribute dispatcher to None
2017-08-16 19:45:13,514 : INFO : loaded lsi.model
2017-08-16 19:45:13,517 : INFO : loading LsiModel object from lsi.model.projection
2017-08-16 19:45:13,607 : INFO : loaded lsi.model.projection


{0.18036565: "It's MacDonald's. .What do you expect?",
 0.38817644: 'Be careful! .I ordered an iced drink @ the drive thru. .The guy '
             '@ the second window never said hi or thanks. .I thought my drink '
             'was just wet on the outside. . Within minutes my .plastic cup '
             'was sticky. .By the time I got home my hands were sticky as well '
             'as my steering wheel, shifter, window controls. .It was a lot of '
             'work getting myself & the car cleaned. .I called the local '
             'office. .All she said was that she was sorry & she would tell '
             'them to wipe off the glasses. .She did not offer to have a '
             'manager call me &/or send me a gift card.I emailed corporate. '
             ".All I have received is a form email .back. .It's been 5 days. "
             '.I think I will stick w/iced-tea lemonades @ Coffee Beans & Tea '
             "Leaves. .Bye MacDonald's!",
 0.43573534: 'One time I was waiting 

## Random Projection Query

In [18]:
result_rpi = temp_Q.rp_query('rpindex.index','mcd.dict','rp.model',rAw_ls)
pprint.pprint(result_rpi)

2017-08-16 19:45:22,428 : INFO : loading MatrixSimilarity object from rpindex.index
2017-08-16 19:45:22,445 : INFO : loaded rpindex.index
2017-08-16 19:45:22,447 : INFO : loading Dictionary object from mcd.dict
2017-08-16 19:45:22,451 : INFO : loaded mcd.dict
2017-08-16 19:45:22,452 : INFO : loading LsiModel object from rp.model
2017-08-16 19:45:22,480 : INFO : loaded rp.model
2017-08-16 19:45:22,482 : INFO : loading LsiModel object from rp.model.projection
2017-08-16 19:45:22,484 : WARNING : failed to load projection from rp.model.projection: [Errno 2] No such file or directory: 'rp.model.projection'


{0.1660313: 'I just had the worst experience ever! First the car behind me was '
            'waited on before me and pulled in front of me. .When I made it to '
            'the cashier I asked if there was an issue with the drive thru he '
            "said no. .He knew what happened but didn't apologize. .When I "
            'reached the window for my mocha I waited five minutes and there '
            'was no ice in the mocha. .The ice had melted while I waited. '
            '.They told me to pull up and they will make me a new one. .After '
            'about ten cars and ten minutes, I left without my mocha!',
 0.26174784: 'One time I was waiting for my meal, about 10 minutes went by, I '
             'asked the cashier where is my foods? She answered someone took '
             'it, I waited another 5 minutes for them to make another one. '
             "This has happened to me twice at this MacDonald, .I don't know "
             'who to blame, the customers who thought there

## TFIDF Query

In [19]:
result_tfidfi = temp_Q.tfidf_query('tfidfindex.index','mcd.dict','tfidf.model',rAw_ls)
pprint.pprint(result_tfidfi)

2017-08-16 19:45:28,458 : INFO : loading MatrixSimilarity object from tfidfindex.index
2017-08-16 19:45:28,518 : INFO : loaded tfidfindex.index
2017-08-16 19:45:28,522 : INFO : loading Dictionary object from mcd.dict
2017-08-16 19:45:28,528 : INFO : loaded mcd.dict
2017-08-16 19:45:28,530 : INFO : loading LsiModel object from tfidf.model
2017-08-16 19:45:28,536 : INFO : loaded tfidf.model
2017-08-16 19:45:28,538 : INFO : loading LsiModel object from tfidf.model.projection
2017-08-16 19:45:28,542 : INFO : loaded tfidf.model.projection


{0.27707428: "worst McDonald's ever!; & i've been to my share of McDonald's. "
             "where do i begin, bad service (I don't have 6 arms), bad manager "
             "(berated chinese employee in front of me), doesn't use "
             "McDonald's merchandise (was given a unwrapped bend-y straw with "
             "my coffee, non-MacDonald's sauce). Worst!",
 0.50638789: 'One time I was waiting for my meal, about 10 minutes went by, I '
             'asked the cashier where is my foods? She answered someone took '
             'it, I waited another 5 minutes for them to make another one. '
             "This has happened to me twice at this MacDonald, .I don't know "
             'who to blame, the customers who thought there is another bag of '
             "food sitting here and just grab it or the cashier who didn't "
             'care the right person get the foods. Anyway, this MacDonald is '
             'one of kind, I went to many other fast foods joints.',
 0.88778734